---
---
# [1] Setup

In [ ]:
from pprint import pp
from pathlib import Path, PurePosixPath
from typing import List

from box_sdk_gen import AddShareLinkToFileSharedLink, UpdateSharedLinkOnFileSharedLink, AddShareLinkToFileSharedLinkPermissionsField, UpdateSharedLinkOnFileSharedLinkPermissionsField # lol

---
## [1.1] Authenticate

In [ ]:
# ''' ———————————————————— Auth via dev token (1hr limit) ———————————————————— '''

# from box_sdk_gen import BoxClient, BoxDeveloperTokenAuth

# token  : str                   = 'VELK7pQGJ1Iz3yQbg9UzGAfkpViTxOTL'
# auth   : BoxDeveloperTokenAuth = BoxDeveloperTokenAuth(token=token)
# client : BoxClient             = BoxClient(auth=auth)

# user_id = client.users.get_user_me().id
# print(f'{user_id = }')

In [ ]:
# ''' ————————————————————————— Auth via OAuth2.0 (?) ———————————————————————— '''

# from box_sdk_gen import BoxOAuth, OAuthConfig

# auth = BoxOAuth(
#     OAuthConfig(client_id="6fzv2qd0x7r70v2i6wunvy6z8v0tw0mn", client_secret="6fzv2qd0x7r70v2i6wunvy6z8v0tw0mn")
# )
# auth_url = auth.get_authorize_url()

# print(auth_url)

In [ ]:
# ''' ————————————————————————————— Auth via JWT ————————————————————————————— '''

# from box_sdk_gen import BoxClient, BoxJWTAuth, JWTConfig

# fpath_jwt_config = list((Path.cwd().parent / '.secret/').glob('*.json'))[0]

# jwt_config = JWTConfig.from_config_file(config_file_path=fpath_jwt_config)
# auth = BoxJWTAuth(config=jwt_config)
# client = BoxClient(auth=auth)

# service_account = client.users.get_user_me()
# print(f"Service Account user ID is {service_account.id}")

In [ ]:
''' —————————————————————————— Auth via JWT (user) ————————————————————————— '''

'''
- NOTE: See /.secret/box_devconsole_app_config.png for howto setup app. Make sure app exists on a developer account (as opposed to a user account that's accessing the dev console — you should create a new account from scratch from the "sign up for Box developer account" page, and you should NOT see a blue textbox at the top of the dev console). Also make sure you "Submit app for authorization for access to the Enterprise" and approve it via the Admin Console. For context, the given example image is from my "iliketochacha1337@gmail.com" account.
'''

from box_sdk_gen import JWTConfig, BoxJWTAuth, BoxClient

fpath_jwt_config : Path       = list((Path.cwd().parent / '.secret/').glob('*.json'))[0]
jwt_config       : JWTConfig  = JWTConfig.from_config_file(config_file_path=fpath_jwt_config)
auth             : BoxJWTAuth = BoxJWTAuth(config=jwt_config)
user_auth        : BoxJWTAuth = auth.with_user_subject(user_id="34633045408")
client           : BoxClient  = BoxClient(auth=user_auth)

---
## [1.2] Test Access

In [ ]:
this_folder_id : str = '270589643170'

this_folder_dict : dict = client.folders.get_folder_by_id(
    folder_id = this_folder_id, 
    limit     = 1_000, 
    offset    = 0,
).to_dict()

pp(this_folder_dict)

---
---
# [2] First try

In [ ]:
# ''' —————————————————————————— Access folder info —————————————————————————— '''

# root_folder_id : str = '270589643170'
# # shared_folder_link : str = 'https://rutgers.box.com/s/ocjo4od5xr9eb0ewjha18fbubbeh2ct2'     
# # # ^ this makes it so it doesn't have to be directly shared in order to access/query
# # # ^^ NVM, although you can access files from just the parent folder share id/link and file id, you don't get a download link. therefore in order to create a registry with download links, you need every file in the folder to be individually shared, and the account with the developer token needs to have the folder shared directly so it can access/modify the files.

# root_folder_dict : dict = client.folders.get_folder_by_id(folder_id=root_folder_id).to_dict()
# # shared_folder : dict = client.folders.get_folder_items(folder_id=shared_folder_id).to_dict()
# ## optional: use 'get_folder_by_id' for more info, or 'get_folder_items' for less info

# pprint.pp(root_folder_dict)

In [ ]:
# subid = root_folder_dict['item_collection']['entries'][0]['id']
# # subid = shared_folder['entries'][0]['id']

# pprint.pp(
#     client.folders.get_folder_by_id(
#     # client.folders.get_folder_items(
#         folder_id=subid,
#     ).to_dict()
# )

In [ ]:
# pprint.pp(
#     client.files.get_file_by_id(
#         file_id='1568654968074',
#         boxapi=f'shared_link={shared_folder_link}',
#     ).to_dict()
# )

In [ ]:
# pprint.pp(
#     client.shared_links_files.get_shared_link_for_file(     # file must be EXPLCIITLY shared for this to work
#         file_id='1568654968074',
#         fields = ["shared_link"],
#     ).to_dict()
# )

---
---
# [3] Recursive Approach SCRAPPING

In [ ]:
# root_folder_id : str = '271398349923'

# root_folder_dict : dict = client.folders.get_folder_by_id(folder_id=root_folder_id).to_dict()

# pp(root_folder_dict)

In [ ]:
# this_file_id : str = '1568776756351'

# this_file_dict : dict = client.files.get_file_by_id(file_id=this_file_id).to_dict()

# pp(this_file_dict)

In [ ]:
# pp(this_file_dict.get('shared_link', None))

In [ ]:
# pp(client.shared_links_files.get_shared_link_for_file(
#     file_id = this_file_id, 
#     fields = "shared_link"
# ).to_dict())

In [ ]:
# pp(client.shared_links_files.add_share_link_to_file(
#     file_id = this_file_id,
#     fields = "shared_link",
#     shared_link = AddShareLinkToFileSharedLink(
#         # access = AddShareLinkToFileSharedLinkAccessField.OPEN.value
#         access = 'open'
#     ),
# ).to_dict())

---
---
# [3.5] too many entries rip

In [ ]:
this_folder_id : str = '271456424795'

this_folder_dict : dict = client.folders.get_folder_by_id(folder_id=this_folder_id, limit=1_000, offset=22_000).to_dict()
# this_folder_dict : dict = client.folders.get_folder_items(folder_id=this_folder_id, limit=1000, offset=20000).to_dict()

pp(this_folder_dict)

---
---
# [4] Recursive Approach FINAL

In [ ]:
def box2registry(
    this_folder_id : str, 
    path_stack     : List[str] = [],
    db             : dict      = {},
) -> dict:
    
    this_folder_dict : dict = client.folders.get_folder_by_id(folder_id=this_folder_id).to_dict()

    path_stack.append(this_folder_dict['name'])


    # NOTE: this_folder_dict['item_collection']['entries'] : List[dict]
    entry : dict

    for entry in this_folder_dict['item_collection']['entries']:
        
        match entry['type']:



            case 'file':
                this_file_id : str = entry['id']
                this_file_dict : dict = client.files.get_file_by_id(file_id=this_file_id).to_dict()
                filepath : PurePosixPath = PurePosixPath(*path_stack, entry['name'])
                this_file_shareinfo_dict : dict = this_file_dict.get('shared_link', None)


                if (this_file_shareinfo_dict is None):
                    print(f'ACTION — adding share link:')
                    print(f'\t- filepath = {filepath}')

                    api_response : dict = client.shared_links_files.add_share_link_to_file(
                        file_id = this_file_id,
                        fields = "shared_link",
                        shared_link = AddShareLinkToFileSharedLink(
                            access = 'open', # AddShareLinkToFileSharedLinkAccessField.OPEN.value
                            password = None,
                            unshared_at = None,
                            permissions = AddShareLinkToFileSharedLinkPermissionsField(
                                can_download = True,
                                can_preview = True,
                                can_edit = False,
                            ),
                        ),
                    ).to_dict()
                    this_file_download_url : str = api_response['shared_link']['download_url']
                    
                    print(f'\t- URL = {this_file_download_url}')


                elif not (
                    this_file_shareinfo_dict['effective_access'] == 'open'
                    and this_file_shareinfo_dict['effective_permission'] == 'can_download'
                    and this_file_shareinfo_dict['is_password_enabled'] == False
                    and this_file_shareinfo_dict['permissions'] == {'can_download':True,'can_preview':True,'can_edit':False}
                    and this_file_shareinfo_dict.get('unshared_at') is None     # NOTE: only paid accounts can change time-until-unshare, so these warnings must be resolved manually.
                ):
                    print()
                    print(f'WARNING — weird share permissions:')
                    print(f'\t- filepath = {filepath}')
                    print(f'\t- Full permissions:')
                    print(f'\t\t{this_file_shareinfo_dict}')
                    print(f'\t- Fixing permissions...')

                    api_response : dict = client.shared_links_files.update_shared_link_on_file(
                        file_id = this_file_id,
                        fields = "shared_link",
                        shared_link = UpdateSharedLinkOnFileSharedLink(
                            access = 'open', 
                            password = None,
                            unshared_at = None,
                            permissions = UpdateSharedLinkOnFileSharedLinkPermissionsField(
                                can_download = True,
                                can_preview = True,
                                can_edit = False,
                            ),
                        ),
                    ).to_dict()
                    this_file_download_url : str = api_response['shared_link']['download_url']
                    
                    print(f'\t- Fixed, hopefully:')
                    print(f'\t\t{api_response["shared_link"]}')
                    print(f'\t- URL = {this_file_download_url}')
                    print()


                else:
                    this_file_download_url : str = this_file_dict['shared_link']['download_url']
                    print(f'File already shared:')
                    print(f'\t- filepath = {filepath}')
                    print(f'\t- URL = {this_file_download_url}')


                db[str(filepath)] = {
                    'id' : this_file_id,
                    'sha1' : entry['sha1'],
                    'download_url' : this_file_download_url,
                }
            


            case 'folder':
                db = box2registry(
                    this_folder_id = entry['id'],
                    path_stack     = path_stack,
                    db             = db,
                )



            case _:
                print()
                print(f'WARNING — Unrecognized entry type:')
                print(f'\t- Full entry:')
                print(f'\t\t{entry}')
                print()

        
    path_stack.pop()

    return db

In [ ]:
root_folder_id : str = '271398349923'
redplanet_registry : dict = box2registry(root_folder_id)

In [ ]:
pp(redplanet_registry)